<a href="https://colab.research.google.com/github/MengOonLee/Web_scraping/blob/master/Mydin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mydin

In [1]:
%%bash

pip install --no-cache-dir -qU pip wheel
pip install --no-cache-dir -qU numpy pandas matplotlib seaborn
pip install --no-cache-dir -qU scrapy
pip check

No broken requirements found.


In [2]:
%%bash

scrapy startproject Mydin

New Scrapy project 'Mydin', using template directory '/home/meng/venv/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /home/meng/work/Web_scraping/Mydin

You can start your first spider with:
    cd Mydin
    scrapy genspider example example.com


In [4]:
%%writefile ./Mydin/venv.sh
pip install --no-cache-dir -U pip wheel
pip install --no-cache-dir -U scrapy
pip check

Writing ./Mydin/venv.sh


In [102]:
from scrapy import Selector
import requests

url = "https://mydinexpress.my/hypermart/product-category/grocery/"
html = requests.get(url).content

sel = Selector(text=html)
products = sel.xpath('//div[contains(@class, "product-grid-item")]')
for p in products:
    title = p.css('h3.product-title::text').extract_first()
    price = ''.join(p.css('span.price ::text').extract())
    print(title, ',', price)
    
next_page = sel.xpath('//div[@class="products-footer"]/a/@href').extract()
print(next_page)

Kismis Golden Raisins (±350g) , RM5.60
Munchy’s Lexus Sandwich Biscuits (456g) – Chocolate Cream ~ , RM8.90
Lay’s Stax Chips (135g) – Sour Cream & Onion ~ , RM5.20
Lay’s Stax Chips (135g) – Barbecue BBQ ~ , RM5.20
Lay’s Stax Chips (135g) – Extra Cheese ~ , RM5.20
Lay’s Stax Chips (135g) – Original ~ , RM5.20
Prego Quick Cook Dried Pasta (71g) – Mac & Cheese , RM4.30
Prego Quick Cook Dried Pasta (71g) – Mushroom Carbonara Mac , RM4.30
Adabi Rempah Gulai Ikan / Fish Savoury Curry Powder (250g) , RM5.30
Lay’s Potato Chips (50g) – Extra BBQ Barbecue ~ , RM3.20
Lay’s Potato Chips (50g) – Hot Chilli Squid ~ , RM3.20
Lay’s Potato Chips (50g) – Original ~ , RM3.20
Lay’s Potato Chips (50g) – Nori Seaweed ~ , RM3.20
Lay’s Potato Chips (46g) – Salted Egg ~ , RM3.20
Maggi Fried Instant Noodle (5 x 75g) – Sambal Tumis Bilis ~ , RM6.40
Kak Sal Sardine in Tomato Sauce (425g) ~ , RM8.80
Kak Sal Sardine in Tomato Sauce with Chilli (425g) ~ , RM8.80
Nureys Laksa Noodle (400g) , RM3.95
ABH Keropok Amplan

In [103]:
%%writefile ./Mydin/Mydin/spiders/grocey_spider.py
import scrapy
import datetime

class GrocerySpider(scrapy.Spider):
    name = 'grocery'
    start_urls = ["https://mydinexpress.my/hypermart/product-category/grocery/"]
    
    def parse(self, response):
        for product in response.xpath('//div[contains(@class, "product-grid-item")]'):
            yield {
                'name': product.css('h3.product-title::text').get(),
                'price': ''.join(product.css('span.price ::text').getall()),
                'datetime': datetime.datetime.now()
            }
            
        for next_page in response.xpath('//div[@class="products-footer"]/a/@href'):
            yield response.follow(next_page, self.parse)

Overwriting ./Mydin/Mydin/spiders/grocey_spider.py


In [104]:
%%writefile ./Mydin/run.sh
#!/bin/bash

scrapy crawl grocery -O ./data/grocey.jl

Overwriting ./Mydin/run.sh
